In [ ]:
import pandas as pd
import numpy as np
import lightgbm
import xgboost as xgb
from tqdm import tqdm
import warnings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [ ]:
def preprocessing(temp_df, pum, len_lag):
    # lag 추가
    for lag in range(1, len_lag+1):
        temp_df[f'p_lag_{lag}'] = -1
        temp_df[f'q_lag_{lag}'] = -1
        for index in range(lag, len(temp_df)):
            temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] # 1일전, 2일전,.. 가격을 feature로 추가
            temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag] # 1일전, 2일전,.. 가격을 feature로 추가

    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date'])
    temp_df['month'] = temp_df['date'].dt.month

    # 예측 대상(1w, 2w, 4w)추가
    for week in ['1_week', '2_week', '4_week']:
        temp_df[week] = 0
        n_week = int(week[0])
        for index in range(len(temp_df)):
            try: temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
            except: continue

    temp_df = temp_df.drop(['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)'], axis=1)

    return temp_df            

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/농산물가격예측/public_data/train.csv')
train.head(2)

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),마늘_거래량(kg),마늘_가격(원/kg),대파_거래량(kg),대파_가격(원/kg),얼갈이배추_거래량(kg),얼갈이배추_가격(원/kg),양배추_거래량(kg),양배추_가격(원/kg),깻잎_거래량(kg),깻잎_가격(원/kg),시금치_거래량(kg),시금치_가격(원/kg),미나리_거래량(kg),미나리_가격(원/kg),당근_거래량(kg),당근_가격(원/kg),파프리카_거래량(kg),파프리카_가격(원/kg),새송이_거래량(kg),새송이_가격(원/kg),팽이버섯_거래량(kg),팽이버섯_가격(원/kg),토마토_거래량(kg),토마토_가격(원/kg),청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,15019.0,5475.0,92334.0,1704.0,6359.0,1331.0,40028.0,348.0,4374.9,13242.0,16550.5,2339.0,10528.0,1729.0,13885.0,804.0,3853.0,3703.0,15797.0,2576.0,14634.0,1474.0,30950.0,1621.0,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0


In [ ]:
pum = '배추'
temp_df = train[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
preprocessing(temp_df, pum, len_lag=28)

,p_lag_1,q_lag_1,p_lag_2,q_lag_2,p_lag_3,q_lag_3,p_lag_4,q_lag_4,p_lag_5,q_lag_5,p_lag_6,q_lag_6,p_lag_7,q_lag_7,p_lag_8,q_lag_8,p_lag_9,q_lag_9,p_lag_10,q_lag_10,p_lag_11,q_lag_11,p_lag_12,q_lag_12,p_lag_13,q_lag_13,p_lag_14,q_lag_14,p_lag_15,q_lag_15,p_lag_16,q_lag_16,p_lag_17,q_lag_17,p_lag_18,q_lag_18,p_lag_19,q_lag_19,p_lag_20,q_lag_20,p_lag_21,q_lag_21,p_lag_22,q_lag_22,p_lag_23,q_lag_23,p_lag_24,q_lag_24,p_lag_25,q_lag_25,p_lag_26,q_lag_26,p_lag_27,q_lag_27,p_lag_28,q_lag_28,month,1_week,2_week,4_week
0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,420,449,625
1,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,389,454,733
2,329.0,80860.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,0,0,1048
3,0.0,0.0,329.0,80860.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,398,475,638
4,478.0,1422742.5,0.0,0.0,329.0,80860.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,431,511,597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,1807.0,2007471.3,1838.0,1757465.6,1813.0,2046286.3,2925.0,1959.0,1812.0,1346091.4,1839.0,1624514.7,1983.0,1459130.1,1939.0,1532777.2,2017.0,1341387.6,2042.0,1221538.6,0.0,0.0,1748.0,968110.0,1576.0,1156811.6,1542.0,1302290.8,1585.0,1181782.2,1994.0,540843.7,1614.0,975020.2,0.0,0.0,1329.0,1104424.8,1358.0,698187.5,1445.0,895628.0,1150.0,1144746.8,1093.0,1279591.6,1133.0,1441152.8,0.0,0.0,1476.0,760499.0,1564.0,763266.0,1561.0,1020033.2,9,0,0,0
1729,1839.0,1856965.0,1807.0,2007471.3,1838.0,1757465.6,1813.0,2046286.3,2925.0,1959.0,1812.0,1346091.4,1839.0,1624514.7,1983.0,1459130.1,1939.0,1532777.2,2017.0,1341387.6,2042.0,1221538.6,0.0,0.0,1748.0,968110.0,1576.0,1156811.6,1542.0,1302290.8,1585.0,1181782.2,1994.0,540843.7,1614.0,975020.2,0.0,0.0,1329.0,1104424.8,1358.0,698187.5,1445.0,895628.0,1150.0,1144746.8,1093.0,1279591.6,1133.0,1441152.8,0.0,0.0,1476.0,760499.0,1564.0,763266.0,9,0,0,0
1730,1789.0,1880095.5,1839.0,1856965.0,1807.0,2007471.3,1838.0,1757465.6,1813.0,2046286.3,2925.0,1959.0,1812.0,1346091.4,1839.0,1624514.7,1983.0,1459130.1,1939.0,1532777.2,2017.0,1341387.6,2042.0,1221538.6,0.0,0.0,1748.0,968110.0,1576.0,1156811.6,1542.0,1302290.8,1585.0,1181782.2,1994.0,540843.7,1614.0,975020.2,0.0,0.0,1329.0,1104424.8,1358.0,698187.5,1445.0,895628.0,1150.0,1144746.8,1093.0,1279591.6,1133.0,1441152.8,0.0,0.0,1476.0,760499.0,9,0,0,0
1731,1760.0,1661090.9,1789.0,1880095.5,1839.0,1856965.0,1807.0,2007471.3,1838.0,1757465.6,1813.0,2046286.3,2925.0,1959.0,1812.0,1346091.4,1839.0,1624514.7,1983.0,1459130.1,1939.0,1532777.2,2017.0,1341387.6,2042.0,1221538.6,0.0,0.0,1748.0,968110.0,1576.0,1156811.6,1542.0,1302290.8,1585.0,1181782.2,1994.0,540843.7,1614.0,975020.2,0.0,0.0,1329.0,1104424.8,1358.0,698187.5,1445.0,895628.0,1150.0,1144746.

In [ ]:
def nmae(week_answer, week_submission):
    answer = np.array(week_answer)
    target_idx = np.where(answer!=0)
    true = answer[target_idx]
    pred = week_submission[target_idx]
    score = np.mean(np.abs(true-pred)/true)

    return score

def at_nmae(pred, dataset):
    y_true = dataset.get_label()
    week_1_answer = y_true[0::3]
    week_2_answer = y_true[1::3]
    week_4_answer = y_true[2::3]
    
    week_1_submission = pred[0::3]
    week_2_submission = pred[1::3]
    week_4_submission = pred[2::3]
    
    score1 = nmae(week_1_answer, week_1_submission)
    score2 = nmae(week_2_answer, week_2_submission)
    score4 = nmae(week_4_answer, week_4_submission)
    
    score = (score1+score2+score4)/3
    
    return 'score', score, False

In [ ]:
def model_train(x_train, y_train, x_valid, y_valid):
    dtrain = xgb.DMatrix(data=x_train, label=y_train)
    dtest = xgb.DMatrix(data=x_valid, label=y_valid)
    
    params = {'learning_rate':0.01,
              'max_depth':6,
              'objective':'reg:squarederror',
              'eval_metric':'mae',
              'subsample': 0.5, 
              'colsample_bytree':0.5,
              'early_stoppings':100,
              'max_leaf_node':100,
              'seed':42,
              'nthread':8,
              'verbose':False}

    model = xgb.train(params=params, dtrain=dtrain, num_boost_round=10000, evals=[(dtrain, 'train'), (dtest, 'eval')], verbose_eval=False)
    
    return model

In [ ]:
unique_pum = ['배추', '무', '양파', '건고추', '마늘',
              '대파', '얼갈이배추', '양배추', '깻잎',
              '시금치', '미나리', '당근',
              '파프리카', '새송이', '팽이버섯', '토마토']

unique_kind = ['청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇']

In [ ]:
model_dict = {}
split = 28 #validation

for pum in tqdm(unique_pum + unique_kind):
    # 품목, 품종별 전처리
    temp_df = train[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']]
    temp_df = preprocessing(temp_df, pum, len_lag=28)
    
    # 주차별 학습
    for week_num in [1, 2, 4]:
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[:,:-3]
        y = temp_df[temp_df[f'{week_num}_week']>0][f'{week_num}_week']

        x_train = x[:-split]
        y_train = y[:-split]
        x_valid = x[-split:]
        y_valid = y[-split:]

        model_dict[f'{pum}_model_{week_num}'] = model_train(x_train, y_train, x_valid, y_valid)

  0%|          | 0/21 [00:00<?, ?it/s]

[15:15:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:17:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:19:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


  5%|▍         | 1/21 [07:37<2:32:32, 457.64s/it]

[15:22:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:25:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[15:27:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


 10%|▉         | 2/21 [15:12<2:24:29, 456.27s/it]

...

 90%|█████████ | 19/21 [2:24:05<15:08, 454.26s/it]

[17:39:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:41:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:43:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


 95%|█████████▌| 20/21 [2:31:19<07:28, 448.11s/it]

[17:46:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:48:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[17:49:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


100%|██████████| 21/21 [2:36:55<00:00, 448.34s/it]


In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/농산물가격예측/sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2020')]['예측대상일자'].str.split('+').str[0].unique()
# ['2020-09-29', ...]

for date in tqdm(public_date_list) :
    test = pd.read_csv(f'/content/drive/MyDrive/농산물가격예측/public_data/test_files/test_{date}.csv')
    for pum in unique_pum + unique_kind:
        # 예측기준일에 대해 전처리
        temp_test = pd.DataFrame([{'date' : date}]) #예측기준일
        alldata = pd.concat([train, test, temp_test], sort=False).reset_index(drop=True)
        alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0)
        alldata = alldata.iloc[-28:].reset_index(drop=True)
        alldata = preprocessing(alldata, pum, len_lag=28)
        temp_test = alldata.iloc[-1].astype(float)
        
        # 개별 모델을 활용하여 1,2,4주 후 가격 예측
        for week_num in [1,2,4] :
            temp_model = model_dict[f'{pum}_model_{week_num}']
            result = temp_model.predict(xgb.DMatrix(pd.DataFrame(temp_test).T.iloc[:, :-3]))
            condition = (submission['예측대상일자']==f'{date}+{week_num}week')
            idx = submission[condition].index
            submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]

100%|██████████| 38/38 [04:09<00:00,  6.57s/it]


In [ ]:
submission.to_csv('/content/drive/MyDrive/농산물가격예측/XGB_baseline.csv',index=False)

In [ ]:
import pandas as pd
xgb = pd.read_csv('/content/drive/MyDrive/농산물가격예측/XGB_baseline.csv')
lgb = pd.read_csv('/content/drive/MyDrive/농산물가격예측/LGBM_baseline.csv')

xgb.iloc[:, 1:] = xgb.iloc[:, 1:]*0.5 + lgb.iloc[:, 1:]*0.5
xgb.head()

,예측대상일자,배추_가격(원/kg),무_가격(원/kg),양파_가격(원/kg),건고추_가격(원/kg),마늘_가격(원/kg),대파_가격(원/kg),얼갈이배추_가격(원/kg),양배추_가격(원/kg),깻잎_가격(원/kg),시금치_가격(원/kg),미나리_가격(원/kg),당근_가격(원/kg),파프리카_가격(원/kg),새송이_가격(원/kg),팽이버섯_가격(원/kg),토마토_가격(원/kg),청상추_가격(원/kg),백다다기_가격(원/kg),애호박_가격(원/kg),캠벨얼리_가격(원/kg),샤인마스캇_가격(원/kg)
0,2020-09-29+1week,1452.181114,885.472141,926.937419,20625.181426,5383.542099,2330.925508,1309.759065,1003.308644,6925.296650,4335.331210,3799.299747,1486.203994,4682.920643,2440.163527,2408.437984,2869.858327,3607.339988,2035.870296,1999.890880,3965.495828,9966.921991
1,2020-09-29+2week,1285.139588,762.473032,933.524642,18008.742713,5438.486073,1897.176330,1101.196845,854.914257,5609.356757,3294.800514,2953.899527,1504.118086,3757.710897,2217.068046,1793.518300,2885.060354,4848.950659,1727.248678,1655.903918,3676.795528,10177.790261
2,2020-09-29+4week,1020.883422,751.422515,916.987377,18798.374337,5405.466719,1409.560733,853.616850,764.848603,5748.790650,3096.752842,2326.088419,1264.317937,3241.167572,2273.681333,1927.600719,2632.880128,5597.427352,1949.586957,1794.782864,3864.800511,9760.456719
3,2020-09-30+1week,1454.273134,870.743824,944.450242,20157.581558,5264.017912,2307.342651,1280.205448,932.934522,7151.166636,3845.438550,3744.387916,1512.136858,4164.465894,2341.028622,2130.453324,2935.366905,3571.819116,1903.355024,1873.792805,3891.994518,10285.036234
4,2020-09-30+2week,1282.030509,797.030075,953.847328,18079.504641,5068.197038,1902.696508,987.353414,824.084554,5415.759174,2986.477139,2966.964748,1392.823702,3693.687016,2172.215019,1813.662221,3186.896810,4340.391833,1655.400737,1672.727161,3550.216872,10562.543799


In [ ]:
xgb.to_csv('/content/drive/MyDrive/농산물가격예측/XGB+LGBM.csv')